In [21]:
from nuscenes.nuscenes import NuScenes
from pyquaternion.quaternion import Quaternion
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from nuscenes.utils.data_classes import PointCloud, LidarPointCloud, RadarPointCloud, Box
import numpy as np
import shutil
from PIL import Image
import os.path as osp
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
from matplotlib import cm
from sphere import Sphere
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from functools import partial
from dh_grid import DHGrid
from sphere import Sphere
from img_sphere import ImageSphere, ColorImageSphere
from metrics import *
from average_meter import AverageMeter
from utils import Utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
# export_ds = '/mnt/data/datasets/nuscenes/processed'
export_ds = '/media/scratch/berlukas/nuscenes'
# sem_clouds_filename = f"{export_ds}/sem_clouds3.npy"
sem_clouds_filename = f"{export_ds}/kitti_processed/sem_clouds_08.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/kitti_processed/sem_clouds_08.npy
Shape of sem clouds is (1000, 3, 200, 200)


In [49]:
n_total_clouds = sem_cloud_features.shape[0]
n_take = 1000

start = n_total_clouds - n_take
end = n_total_clouds

val_features = np.copy(sem_cloud_features[start:end, :, :, :])
print(f"Shape of val clouds is {val_features.shape}")

Shape of val clouds is (1000, 3, 200, 200)


In [50]:
np.random.shuffle(val_features)

n_subset = 200
subset_feature = val_features[0:n_subset, :, :, :]
print(f"Shape of subset clouds is {subset_feature.shape}")

filename = f"{export_ds}/sem_clouds_val_kitti_final.npy"
np.save(filename, subset_feature)
print(f"Wrote subset features to {filename}.")

Shape of subset clouds is (200, 3, 200, 200)
Wrote subset features to /media/scratch/berlukas/nuscenes/sem_clouds_val_kitti_final.npy.


In [52]:
print(np.unique(val_features[100,1,:,:]))

[ -1.   0.   2.   5.   7.  10.  12.  15.  17.  20.  22.  25.  28.  30.
  33.  35.  38.  40.  43.  45.  48.  51.  53.  56.  58.  61.  63.  66.
  68.  71.  73.  76.  79.  81.  84.  86.  89.  91.  94.  96.  99. 102.
 104. 107. 109. 112. 114. 117. 119. 122. 124. 127. 130. 132. 135. 137.
 140. 142. 145. 147. 150. 153. 155. 158. 160. 163. 165. 168. 170. 173.
 175. 178. 181. 183. 186. 193. 201. 204. 206. 211. 221. 232. 237. 252.]
